<a href="https://colab.research.google.com/github/Arun-nexus/computer_vision/blob/main/animal_prediction_day1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d sharansmenon/animals141

Dataset URL: https://www.kaggle.com/datasets/sharansmenon/animals141
License(s): copyright-authors
animals141.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import zipfile
with zipfile.ZipFile("animals141.zip","r") as zip:
  zip.extractall("animal")

In [5]:
from torchvision import transforms

In [6]:
augmentation=transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

In [7]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

In [8]:
dataset="/content/animal/dataset/dataset"
training_data = ImageFolder(root=dataset,transform=augmentation)

In [9]:
train_size=int(0.7*len(training_data))
val_size=len(training_data)-train_size

In [10]:
from torch.utils.data import random_split

In [11]:
train_data,val_data=random_split(training_data,[train_size,val_size])

In [22]:
train_loader=DataLoader(train_data,shuffle=True,batch_size=512,num_workers=2)
val_loader=DataLoader(val_data,batch_size=512,num_workers=2)

In [13]:
from torch import nn

In [14]:
import torchvision.models as models

In [15]:
class animal_cnn(nn.Module):
  def __init__(self,num_classes):
    super(animal_cnn,self).__init__()
    self.vgg19=models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)
    for param in self.vgg19.features.parameters():
      param.requires_grad=False

    self.vgg19.classifier=nn.Sequential(
      nn.Linear(25088,4096),
      nn.ReLU(),
      nn.Dropout(),
      nn.Linear(4096,1024),
      nn.ReLU(),
      nn.Dropout(),
      nn.Linear(1024,151)
    )
  def forward(self,x):
    return self.vgg19(x)

In [16]:
import torch

In [17]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [18]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [19]:
model=animal_cnn(num_classes=151).to(device)
model_loss=nn.CrossEntropyLoss()
optim=torch.optim.Adam(model.parameters(),lr=1e-4)
scheduler=ReduceLROnPlateau(optim,mode="min",patience=2)

In [20]:
class earlystopping():
  def __init__(self,min_delta,patience=3):
    self.patience=patience
    self.min_delta=min_delta
    self.early_stop=False
    self.best_loss=float("inf")
    self.counter=0

  def __call__(self,val_loss):
    if val_loss < self.best_loss - self.min_delta:
      self.best_loss=val_loss
      self.counter=0
    else:
      self.counter+=1
      if self.counter>=self.patience:
        self.early_stop=True

In [23]:
epochs = 50
training_loss = []
accuracy = []
val_losses = []
val_accuracy = []

earlystopping_callback = earlystopping(min_delta=0.01, patience=5)

for epoch in range(epochs):
    model.train()
    running_loss = 0
    mcorrect, mtotal = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = model_loss(outputs, labels)

        optim.zero_grad()
        loss.backward()
        optim.step()

        running_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        mcorrect += (preds == labels).sum().item()
        mtotal += labels.size(0)

    train_loss = running_loss / len(train_loader)
    acc = 100 * mcorrect / mtotal
    training_loss.append(train_loss)
    accuracy.append(acc)

    model.eval()
    val_running_loss = 0
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            v_loss = model_loss(outputs, labels)

            val_running_loss += v_loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    val_loss = val_running_loss / len(val_loader)
    val_acc = 100 * correct / total
    val_losses.append(val_loss)
    val_accuracy.append(val_acc)

    print(f"Epoch {epoch+1}: Train Acc={acc:.2f}%, Train Loss={train_loss:.4f} | Val Acc={val_acc:.2f}%, Val Loss={val_loss:.4f}")

    earlystopping_callback(val_loss)
    if earlystopping_callback.early_stop:
        print("Early stopping triggered")
        break

Epoch 1: Train Acc=6.61%, Train Loss=4.7260 | Val Acc=20.73%, Val Loss=4.2564
Epoch 2: Train Acc=17.25%, Train Loss=4.0071 | Val Acc=31.15%, Val Loss=3.5068
Epoch 3: Train Acc=25.93%, Train Loss=3.4521 | Val Acc=38.22%, Val Loss=2.9750
Epoch 4: Train Acc=35.25%, Train Loss=2.9556 | Val Acc=43.17%, Val Loss=2.6144
Epoch 5: Train Acc=40.26%, Train Loss=2.6115 | Val Acc=49.60%, Val Loss=2.3370
Epoch 6: Train Acc=45.45%, Train Loss=2.3367 | Val Acc=51.36%, Val Loss=2.1787
Epoch 7: Train Acc=48.39%, Train Loss=2.1590 | Val Acc=52.84%, Val Loss=2.0701
Epoch 8: Train Acc=52.29%, Train Loss=1.9763 | Val Acc=54.44%, Val Loss=1.9654
Epoch 9: Train Acc=56.07%, Train Loss=1.8053 | Val Acc=54.60%, Val Loss=1.9012
Epoch 10: Train Acc=59.13%, Train Loss=1.6811 | Val Acc=56.14%, Val Loss=1.8098
Epoch 11: Train Acc=59.10%, Train Loss=1.6339 | Val Acc=56.94%, Val Loss=1.8050
Epoch 12: Train Acc=62.91%, Train Loss=1.5213 | Val Acc=57.04%, Val Loss=1.7397
Epoch 13: Train Acc=64.02%, Train Loss=1.4417 | Va

In [24]:
import torch

torch.save(model, "animal_cnn_vgg19.pth")


In [26]:
from google.colab import files

files.download("animal_cnn_vgg19.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
new_data=files.upload()

In [ ]:
from PIL import Image

In [ ]:
new_image=Image.open("/content/download (1).jpg")

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(new_image)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

new_image_tensor =(transform(new_image).unsqueeze(0)).to(device)

In [ ]:
model.eval()
with torch.no_grad():
  out=model(new_image_tensor)
  idx=torch.max(out,1).indices.item()
name=training_data.classes[idx]

In [ ]:
import json

file_path = "/content/animal/translation.json"

with open(file_path, 'r') as f:
  translation_data = json.load(f)
  dictionary=translation_data
  print(dictionary)

In [ ]:
for i in dictionary:
  if i==name:
    print(dictionary[i])
